In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import json
import string
import nltk
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Flatten, GlobalMaxPooling1D
from tensorflow.keras.models import Model
import random

In [2]:
intents = json.loads(open('intents.json').read())

In [3]:
intents

{'intents': [{'tag': 'salam',
   'patterns': ['Halo',
    'Apa kabar?',
    'Ada yang bisa saya bantu?',
    'Hai',
    'Hello',
    'Selamat siang'],
   'responses': ['Halo, terima kasih telah bertanya',
    'Selamat datang di Kebun Binatang, bagaimana saya bisa membantu Anda?',
    'Hai, ada yang bisa saya jelaskan tentang kebun binatang?']},
  {'tag': 'selamat tinggal',
   'patterns': ['Selamat tinggal',
    'Sampai jumpa nanti',
    'Sampai jumpa',
    'Terima kasih atas kunjungannya, sampai jumpa nanti',
    'Sampai ketemu lain waktu'],
   'responses': ['Sampai jumpa!',
    'Semoga Anda memiliki waktu yang menyenangkan di kebun binatang.',
    'Selamat tinggal! Kembali lagi nanti ya.']},
  {'tag': 'terima kasih',
   'patterns': ['Terima kasih',
    'Makasih ya',
    'Bantuanku sangat berguna',
    'Keren, terima kasih',
    'Terima kasih atas bantuanmu'],
   'responses': ['Senang bisa membantu!', 'Kapan saja!', 'Sama-sama']},
  {'tag': 'tanpa jawaban',
   'patterns': [],
   'respo

In [8]:
tags = []
inputs = []
responses = {}
for intent in intents['intents']:
    responses[intent['tag']] = intent["responses"]
    for lines in intent['patterns']:
        inputs.append(lines)
        tags.append(intent['tag'])


print("this is tags",tags)
print("this is input",inputs)
print("this is responses",responses)

this is tags ['salam', 'salam', 'salam', 'salam', 'salam', 'salam', 'selamat tinggal', 'selamat tinggal', 'selamat tinggal', 'selamat tinggal', 'selamat tinggal', 'terima kasih', 'terima kasih', 'terima kasih', 'terima kasih', 'terima kasih', 'pilihan', 'pilihan', 'pilihan', 'pilihan', 'pilihan', 'konservasi_hewan', 'konservasi_hewan', 'konservasi_hewan', 'konservasi_hewan', 'konservasi_hewan', 'fakta_konservasi', 'fakta_konservasi', 'fakta_konservasi', 'fakta_konservasi', 'fakta_konservasi']
this is input ['Halo', 'Apa kabar?', 'Ada yang bisa saya bantu?', 'Hai', 'Hello', 'Selamat siang', 'Selamat tinggal', 'Sampai jumpa nanti', 'Sampai jumpa', 'Terima kasih atas kunjungannya, sampai jumpa nanti', 'Sampai ketemu lain waktu', 'Terima kasih', 'Makasih ya', 'Bantuanku sangat berguna', 'Keren, terima kasih', 'Terima kasih atas bantuanmu', 'Bagaimana Anda bisa membantu saya?', 'Apa yang bisa Anda lakukan?', 'Apa bantuan yang Anda tawarkan?', 'Bagaimana Anda bisa membantu?', 'Apa dukungan y

In [9]:
data = pd.DataFrame({"inputs":inputs,
                    "tags":tags})

In [10]:
data

,inputs,tags
0,Halo,salam
1,Apa kabar?,salam
2,Ada yang bisa saya bantu?,salam
3,Hai,salam
4,Hello,salam
5,Selamat siang,salam
6,Selamat tinggal,selamat tinggal
7,Sampai jumpa nanti,selamat tinggal
8,Sampai jumpa,selamat tinggal
9,"Terima kasih atas kunjungannya, sampai jumpa n...",selamat tinggal


Removing punctuations and converting text to lower case

In [11]:
data["inputs"] = data["inputs"].apply(lambda wrd:[ltrs.lower() for ltrs in wrd if ltrs not in string.punctuation])

In [12]:
data["inputs"] = data["inputs"].apply(lambda wrd: ''.join(wrd))

In [13]:
data

,inputs,tags
0,halo,salam
1,apa kabar,salam
2,ada yang bisa saya bantu,salam
3,hai,salam
4,hello,salam
5,selamat siang,salam
6,selamat tinggal,selamat tinggal
7,sampai jumpa nanti,selamat tinggal
8,sampai jumpa,selamat tinggal
9,terima kasih atas kunjungannya sampai jumpa nanti,selamat tinggal


In [14]:
tokenizer = Tokenizer(num_words = 2000)

In [15]:
tokenizer.fit_on_texts(data["inputs"])

In [16]:
train = tokenizer.texts_to_sequences(data["inputs"])

In [17]:
train

[[24],
 [1, 25],
 [16, 2, 3, 4, 26],
 [27],
 [28],
 [17, 29],
 [17, 30],
 [8, 13, 18],
 [8, 13],
 [9, 10, 19, 31, 8, 13, 18],
 [8, 32, 33, 34],
 [9, 10],
 [35, 36],
 [37, 38, 39],
 [40, 9, 10],
 [9, 10, 19, 41],
 [14, 11, 3, 20, 4],
 [1, 2, 3, 11, 21],
 [1, 42, 2, 11, 43],
 [14, 11, 3, 20],
 [1, 44, 2, 45],
 [46, 5, 6, 7],
 [22, 23, 47, 5, 6, 7],
 [14, 4, 3, 48, 6],
 [49, 6, 7, 1, 2, 16, 50, 51, 52],
 [1, 2, 3, 4, 21, 53, 54, 7, 55],
 [22, 12, 56, 5, 6],
 [12, 57, 5, 15, 7],
 [58, 4, 23, 5, 15, 7],
 [59, 4, 12, 6],
 [12, 60, 5, 61, 15]]

In [18]:
x_train = pad_sequences(train)

In [19]:
x_train

array([[ 0,  0,  0,  0,  0,  0,  0,  0, 24],
       [ 0,  0,  0,  0,  0,  0,  0,  1, 25],
       [ 0,  0,  0,  0, 16,  2,  3,  4, 26],
       [ 0,  0,  0,  0,  0,  0,  0,  0, 27],
       [ 0,  0,  0,  0,  0,  0,  0,  0, 28],
       [ 0,  0,  0,  0,  0,  0,  0, 17, 29],
       [ 0,  0,  0,  0,  0,  0,  0, 17, 30],
       [ 0,  0,  0,  0,  0,  0,  8, 13, 18],
       [ 0,  0,  0,  0,  0,  0,  0,  8, 13],
       [ 0,  0,  9, 10, 19, 31,  8, 13, 18],
       [ 0,  0,  0,  0,  0,  8, 32, 33, 34],
       [ 0,  0,  0,  0,  0,  0,  0,  9, 10],
       [ 0,  0,  0,  0,  0,  0,  0, 35, 36],
       [ 0,  0,  0,  0,  0,  0, 37, 38, 39],
       [ 0,  0,  0,  0,  0,  0, 40,  9, 10],
       [ 0,  0,  0,  0,  0,  9, 10, 19, 41],
       [ 0,  0,  0,  0, 14, 11,  3, 20,  4],
       [ 0,  0,  0,  0,  1,  2,  3, 11, 21],
       [ 0,  0,  0,  0,  1, 42,  2, 11, 43],
       [ 0,  0,  0,  0,  0, 14, 11,  3, 20],
       [ 0,  0,  0,  0,  0,  1, 44,  2, 45],
       [ 0,  0,  0,  0,  0, 46,  5,  6,  7],
       [ 0

Encoding the output

In [20]:
le = LabelEncoder()

In [21]:
y_train = le.fit_transform(data["tags"])

In [22]:
y_train

array([3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 2, 2, 2, 2, 2, 1,
       1, 1, 1, 1, 0, 0, 0, 0, 0])

In [23]:
input_shape = x_train.shape[1]
print(input_shape)

9


In [24]:
unique_words = len(tokenizer.word_index)
output_length = le.classes_.shape[0]
print("unique words: ", unique_words)
print("output length: ", output_length)

unique words:  61
output length:  6


##Building Neural Network
Here I am using Embedding layer and LSTM (Long Short term Memory) and 3 dense layers. Since we are predicting tags, so the output length will be the number of classes (tags), therefore using softmax activation in the last layer.

In [25]:
model = tf.keras.Sequential()
model.add(Input(shape=(input_shape,)))
model.add(Embedding(unique_words+1, 10, input_length= (input_shape,)))
model.add(LSTM(10, return_sequences=True))
model.add(Flatten())
model.add(Dense(units= 10, activation='relu'))
model.add(Dense(units= 5, activation='relu'))
model.add(Dense(units= output_length, activation='softmax'))

In [26]:
model.compile(loss = "sparse_categorical_crossentropy", optimizer ='adam', metrics=['accuracy'])

Training the Model

In [27]:
jarvis = model.fit(x_train, y_train, epochs=300)

Epoch 1/300
1/1 [==============================] - 4s 4s/step - loss: 1.7932 - accuracy: 0.1290
Epoch 2/300
1/1 [==============================] - 0s 16ms/step - loss: 1.7921 - accuracy: 0.1290
Epoch 3/300
1/1 [==============================] - 0s 16ms/step - loss: 1.7914 - accuracy: 0.1935
Epoch 4/300
1/1 [==============================] - 0s 17ms/step - loss: 1.7912 - accuracy: 0.2581
Epoch 5/300
1/1 [==============================] - 0s 20ms/step - loss: 1.7908 - accuracy: 0.2581
Epoch 6/300
1/1 [==============================] - 0s 17ms/step - loss: 1.7904 - accuracy: 0.2581
Epoch 7/300
1/1 [==============================] - 0s 22ms/step - loss: 1.7900 - accuracy: 0.2581
Epoch 8/300
1/1 [==============================] - 0s 18ms/step - loss: 1.7896 - accuracy: 0.2903
Epoch 9/300
1/1 [==============================] - 0s 17ms/step - loss: 1.7892 - accuracy: 0.2903
Epoch 10/300
1/1 [==============================] - 0s 16ms/step - loss: 1.7888 - accuracy: 0.2903
Epoch 11/300
1/1 [===

Testing

In [33]:
#Testing

while True:
    textList = []
    user_input = input("Kamu: ")
    prediction_input = []

    for letter in user_input:
        if letter not in string.punctuation:
            prediction_input.append(letter.lower())

    prediction_input = ''.join(prediction_input)
    textList.append(prediction_input)

    prediction_input = tokenizer.texts_to_sequences(textList)
    prediction_input = np.array(prediction_input).reshape(-1)
    prediction_input = pad_sequences([prediction_input], input_shape)

    output = model.predict(prediction_input)
    output = output.argmax()

    response_tag = le.inverse_transform([output])[0]
    print("Jarvis: ", random.choice(responses[response_tag]))
    if response_tag == 'Selamat Tinggal':
        break

KeyboardInterrupt: ignored

In [ ]:
import pickle
pickle.dump(rf,open('model1.pkl','wb'))